In [ ]:
!mkdir -p /etc/apt/keyrings; mkdir -p ~/.gnupg; chmod 700 ~/.gnupg
!gpg --no-default-keyring --keyring /etc/apt/keyrings/skewed.de.gpg --keyserver keyserver.ubuntu.com --recv-keys 612DEFB798507F25
!echo "deb [signed-by=/etc/apt/keyrings/skewed.de.gpg] https://downloads.skewed.de/apt $(lsb_release -s -c) main" > /etc/apt/sources.list.d/skewed.list
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

# Colab uses a Python install that deviates from the system's! Bad colab! We need some workarounds.
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

In [ ]:
import pymc as pm
import numpy as np
import pandas as pd
import graph_tool.all as gt
import matplotlib.pyplot as plt
import arviz as az  # For trace plotting
import pytensor.tensor as pt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Load the data graph
g = gt.collection.ns["board_directors/net2m_2002-05-01"]

# Map the gender values to labels for community assignment (3 groups: 0, 1, 2)
gender_property = g.vertex_properties['gender']
q_actual = np.array([int(gender_property[v]) for v in g.vertices()])

# Create adjacency matrix W based on the edges in the graph
n = g.num_vertices()
W = np.zeros((n, n))
for e in g.edges():
    i, j = int(e.source()), int(e.target())
    W[i, j] = W[j, i] = 1

# Set q_observed based on a known fraction of labels
known_fraction = 0.9
known_size = int(known_fraction * n)
unknown_indices = np.random.choice(n, n - known_size, replace=False)
q_observed = q_actual.copy()
q_observed[unknown_indices] = -1  # Set unknown nodes to -1

# Calculate the actual assortativity coefficient r_actual
H = np.zeros((n, 3))
H[q_actual == 0, 0] = 1
H[q_actual == 1, 1] = 1
H[q_actual == 2, 2] = 1
e_matrix = H.T @ W @ H
e_matrix /= e_matrix.sum()

a_actual = np.sum(e_matrix, axis=1)
b_actual = np.sum(e_matrix, axis=0)
sum_eii_actual = np.trace(e_matrix)
sum_aibi_actual = np.sum(a_actual * b_actual)
r_actual = (sum_eii_actual - sum_aibi_actual) / (1 - sum_aibi_actual)

print(f"Actual assortativity coefficient r for n={n}: {r_actual}")

# Get the indices of known labels
known_indices = np.where(q_observed != -1)[0]
observed_labels = q_observed[known_indices]
unknown_indices = np.where(q_observed == -1)[0]

# Define the PyMC Stochastic Block Model with 3 groups
with pm.Model() as sbm_model:
    # Define the Dirichlet prior distribution for community proportions with 3 groups
    f = pm.Dirichlet('f', a=np.array([1.0, 1.0, 1.0]))

    # Handle known labels as observed values in the model
    g_known = pm.Categorical('g_known', p=f, shape=len(known_indices), observed=observed_labels)

    # Treat unknown labels as latent variables to be inferred
    g_unknown = pm.Categorical('g_unknown', p=f, shape=n - len(known_indices))

    # Define the Beta prior for edge probabilities with 3x3 matrix
    p = pm.Beta('p', alpha=10, beta=1, shape=(3, 3))

    # Define the likelihood based on the adjacency matrix W for known and unknown nodes separately
    A_known = pm.Bernoulli('A_known', p=p[g_known[:, None], g_known[None, :]], observed=W[known_indices][:, known_indices])
    A_unknown = pm.Bernoulli('A_unknown', p=p[g_unknown[:, None], g_unknown[None, :]], observed=W[unknown_indices][:, unknown_indices])

    # Calculate assortativity coefficient r in the model
    e_matrix = f[:, None] * p * f[None, :]
    e_matrix /= pt.sum(e_matrix)

    a = pt.sum(e_matrix, axis=1)
    b = pt.sum(e_matrix, axis=0)

    sum_eii = pt.sum(pt.diag(e_matrix))
    sum_aibi = pt.sum(a * b)
    r = pm.Deterministic('r', (sum_eii - sum_aibi) / (1 - sum_aibi))

    # Sample from the posterior
    trace = pm.sample(3000, tune=1000, target_accept=0.95)

# Plot the trace for all variables in the model
az.plot_trace(trace)
plt.show()

# Extract and display the trace results
r_inferred_samples = trace.posterior['r'].values.flatten()  # Convert to a flat NumPy array

# Calculate observed assortativity coefficient r_observed
def calculate_observed_r(W, observed_q):
    """Calculate the assortativity coefficient r based on observed group labels."""
    observed_indices = np.where(observed_q != -1)[0]
    W_observed = W[np.ix_(observed_indices, observed_indices)]
    H = np.zeros((len(observed_q[observed_q != -1]), 3))
    H[observed_q[observed_q != -1] == 0, 0] = 1
    H[observed_q[observed_q != -1] == 1, 1] = 1
    H[observed_q[observed_q != -1] == 2, 2] = 1

    e_matrix = H.T @ W_observed @ H
    e_matrix /= e_matrix.sum()

    a_observed = np.sum(e_matrix, axis=1)
    b_observed = np.sum(e_matrix, axis=0)

    sum_eii_observed = np.trace(e_matrix)
    sum_aibi_observed = np.sum(a_observed * b_observed)
    r_observed = (sum_eii_observed - sum_aibi_observed) / (1 - sum_aibi_observed)

    return r_observed

r_observed = calculate_observed_r(W, q_observed)
r_inferred = np.mean(r_inferred_samples)

# Plot the posterior distribution of assortativity coefficient (r)
plt.figure(figsize=(8, 6))
plt.hist(r_inferred_samples, bins=30, density=True, alpha=0.7, color='blue', label='inferred r histogram')
plt.axvline(r_actual, color='green', linestyle='--', label='true r')
plt.axvline(r_observed, color='red', linestyle='-.', label='observed r')
plt.axvline(r_inferred, color='purple', linestyle=':', label='inferred r')

# Add annotations showing the actual, observed, and inferred r values
plt.text(0.6, 10, f"true r = {r_actual:.4f}\nobserved r = {r_observed:.4f}\ninferred r = {r_inferred:.4f}",
         fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

plt.title('Posterior Distribution of Assortativity Coefficient (r)')
plt.xlabel('Assortativity Coefficient (r)')
plt.ylabel('Density')
plt.legend()
plt.show()

# Step 1: Extract the predicted labels for the unknown nodes from the trace
g_unknown_samples = trace.posterior['g_unknown'].values
g_unknown_predicted = np.mean(g_unknown_samples, axis=(0, 1)).round().astype(int)  # Take the mean and round it to get predicted labels

# Step 2: Get the actual labels for the unknown nodes
q_actual_unknown = q_actual[unknown_indices]

# Step 3: Create a pandas DataFrame to show the comparison
unknown_label_comparison = pd.DataFrame({
    'Node Index': unknown_indices,
    'Predicted Label': g_unknown_predicted,
    'Actual Label': q_actual_unknown
})

# Display the table using pandas
print(unknown_label_comparison)